In [2]:
import pandas as pd
import re 
import numpy as np

# Define the file name (ensure the file is in your working directory or provide the full path)
file_name = "data/S001_u7.5.2024_DI_from_3.2024_Merged_Demo_Geo_by_Tithi.xlsx"

# Specify the columns to be read from the Excel file
columns_to_use = [
    'ID', 
    'GENDER', 
    'demo_age', 
    'PRIMARY_RACE', 
    'LANGUAGE', 
    'PRIMARY_ETHNICITY', 
    'D_Insur_at_pull', 
    'RPL_THEME1'
]

# Read the Excel file using pandas; if the file has multiple sheets, you may need to specify the sheet_name parameter
df = pd.read_excel(file_name, usecols=columns_to_use)


df['PRIMARY_RACE'] = df['PRIMARY_RACE'].replace(
    ["Declined / Not Available", "Choose not to Answer", "Unknown", ''],
    'Unknown'
)

# Replace NaN values with "unknown"
df['LANGUAGE'] = df['LANGUAGE'].fillna('Unknown')

# Replace blank strings (including those that are only whitespace) with "unknown"
df['LANGUAGE'] = df['LANGUAGE'].replace(r'^\s*$', 'Unknown', regex=True)

df['PRIMARY_ETHNICITY'] = df['PRIMARY_ETHNICITY'].replace(
    ["Patient Refused", "Patient chooses not to answer", "Unknown/Not Specified",""],
    'Unknown'
)

# Replace NaN values with "unknown"
df['D_Insur_at_pull'] = df['D_Insur_at_pull'].fillna('Unknown')

# Replace blank strings (including those that are only whitespace) with "unknown"
df['D_Insur_at_pull'] = df['D_Insur_at_pull'].replace(r'^\s*$', 'Unknown', regex=True)



# Define which columns are categorical and need one hot encoding
categorical_columns = ['GENDER', 'PRIMARY_RACE', 'LANGUAGE', 'PRIMARY_ETHNICITY', 'D_Insur_at_pull']


# # Apply one hot encoding to the categorical variables
df_encoded = pd.get_dummies(df, columns=categorical_columns)

"""Replaces negative values, values > 1, and non-numeric values in 'RPL_THEME1' with NaN."""
df_encoded["RPL_THEME1"] = pd.to_numeric(df_encoded["RPL_THEME1"], errors="coerce")  # Convert non-numeric to NaN
df_encoded.loc[(df_encoded["RPL_THEME1"] < 0) | (df_encoded["RPL_THEME1"] > 1), "RPL_THEME1"] = np.nan  # Replace invalid values


# Load the Excel spreadsheet
data_path = "data/anonymized_H-43413 Qualifying Encounters.xlsx"
df_icd10 = pd.read_excel(data_path)

# List of ICD-10 codes to check
# icd10_codes = [
#     "F20.9", "F25.0", "F25.9", "F22", "F20.3", "F29", "F20.89", "F20.0", "F25.8", "F23", 
#     "F20.2", "F20.1", "F25.1", "F20.5", "F28", "F21", "F20.81", "F24"
# ]

icd10_codes = ['F28', 'F20.5', 'F20.2', 'F21', 'F23', 'F24', 'F20.81', 'F25.8', 'F20.1']

# Create binary columns for each ICD-10 code
df_icd10_binary = df_icd10.pivot_table(index='ID', columns='ICD-10', aggfunc='size', fill_value=0)

# Ensure all specified ICD-10 codes are present in the dataframe
df_icd10_binary = df_icd10_binary.reindex(columns=icd10_codes, fill_value=0)

# Reset index to bring ID back as a column
df_icd10_binary.reset_index(inplace=True)

# Merge with df_encoded
df_encoded = df_encoded.merge(df_icd10_binary, on="ID", how="left")

# Fill NaN values with 0 (for IDs that did not appear in the ICD-10 dataset)
df_encoded.fillna(0, inplace=True)

# Convert to binary: if value is 1 or more, set to 1, otherwise keep 0
df_encoded[icd10_codes] = (df_encoded[icd10_codes] > 0).astype(int)





# Load the Excel file
file_path = "data/anonymized_H-43413 Amb Med Data.xlsx"
df_med = pd.read_excel(file_path)

# Define medication keywords (partial match allowed)
medications = ["CLOZAPINE", "RISPERIDONE", "OLANZAPINE"]

# Standardize medication names (uppercase for uniformity)
df_med["MEDICATION NAME"] = df_med["MEDICATION NAME"].str.upper()

# Function to check if any medication keyword is in the "MEDICATION NAME"
def contains_medication(med_name, target_list):
    return any(target in med_name for target in target_list)

# Filter only rows where "MEDICATION NAME" contains any target medication
df_filtered = df_med[df_med["MEDICATION NAME"].apply(lambda x: contains_medication(x, medications))].copy()

# Function to extract numeric dose values from strings like "10 MG"
def extract_numeric_dose(dose):
    if isinstance(dose, str):
        match = re.search(r"[\d]+(?:\.\d+)?", dose)  # Extract number including decimals
        return float(match.group()) if match else None
    return dose  # If already a number, return as is

# Apply extraction function to clean "DOSE" column
df_filtered["DOSE"] = df_filtered["DOSE"].apply(extract_numeric_dose)

# Drop rows where DOSE couldn't be extracted
df_filtered = df_filtered.dropna(subset=["DOSE"])

# Convert DOSE to numeric
df_filtered["DOSE"] = df_filtered["DOSE"].astype(float)

# Map full medication names back to their generic form (CLOZAPINE, RISPERIDONE, OLANZAPINE)
def get_medication_generic_name(med_name):
    for med in medications:
        if med in med_name:
            return med
    return None

df_filtered["GENERIC_MED_NAME"] = df_filtered["MEDICATION NAME"].apply(get_medication_generic_name)

# Aggregate statistics per ID and Medication
df_stats = df_filtered.groupby(["ID", "GENERIC_MED_NAME"])["DOSE"].agg(["min", "max", "mean"]).reset_index()

# Pivot table to create separate columns for each medication and statistic
df_pivot = df_stats.pivot(index="ID", columns="GENERIC_MED_NAME", values=["min", "max", "mean"])

# Rename columns for clarity (e.g., min_CLOZAPINE, max_CLOZAPINE, mean_CLOZAPINE)
df_pivot.columns = [f"{stat}_{med}" for stat, med in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Ensure all medication dosage variables exist in df_encoded, even if missing in df_pivot
for med in medications:
    for stat in ["min", "max", "mean"]:
        col_name = f"{stat}_{med}"
        if col_name not in df_pivot.columns:
            df_pivot[col_name] = 0  # Add missing columns with 0

# Merge with df_encoded
df_encoded = df_encoded.merge(df_pivot, on="ID", how="left")

# Replace NaN values in the 9 medication-related columns with 0
for med in medications:
    for stat in ["min", "max", "mean"]:
        col_name = f"{stat}_{med}"
        df_encoded[col_name] = df_encoded[col_name].fillna(0)

# Load the vitals CSV file
vitals_file_path = "data/anonymized_H43413_vitals.csv"
df_vitals = pd.read_csv(vitals_file_path)

# Standardize FLO_DISPLAY_NM (uppercase for uniformity)
df_vitals["FLO_DISPLAY_NM"] = df_vitals["FLO_DISPLAY_NM"].str.upper()

# Define relevant vitals (case-insensitive search)
vitals = ["HEIGHT", "WEIGHT", "BMI", "PULSE"]

# Function to check if any vital keyword is in "FLO_DISPLAY_NM"
def contains_vital(vital_name, target_list):
    return any(target in vital_name for target in target_list)

# Filter only rows where "FLO_DISPLAY_NM" contains relevant vitals
df_filtered = df_vitals[df_vitals["FLO_DISPLAY_NM"].apply(lambda x: contains_vital(x, vitals))].copy()

# Function to extract numeric values from vitals data
def extract_numeric_value(value):
    if isinstance(value, str):
        match = re.search(r"[\d]+(?:\.\d+)?", value)  # Extract number including decimals
        return float(match.group()) if match else None
    return value  # If already a number, return as is

# Apply extraction function to clean values
df_filtered["ENTERED_VALUE"] = df_filtered["ENTERED_VALUE"].apply(extract_numeric_value)

# Drop rows where VALUE couldn't be extracted
df_filtered = df_filtered.dropna(subset=["ENTERED_VALUE"])

# Convert VALUE to numeric
df_filtered["ENTERED_VALUE"] = df_filtered["ENTERED_VALUE"].astype(float)

# Map full FLO_DISPLAY_NM names back to generic names (HEIGHT, WEIGHT, BMI)
def get_vital_name(display_name):
    for vital in vitals:
        if vital in display_name:
            return vital
    return None

df_filtered["VITAL_NAME"] = df_filtered["FLO_DISPLAY_NM"].apply(get_vital_name)

# Aggregate statistics per ID and Vital
df_stats = df_filtered.groupby(["ID", "VITAL_NAME"])["ENTERED_VALUE"].agg(["min", "max", "mean"]).reset_index()

# Pivot table to create separate columns for each vital and statistic
df_pivot = df_stats.pivot(index="ID", columns="VITAL_NAME", values=["min", "max", "mean"])

# Rename columns for clarity (e.g., min_HEIGHT, max_HEIGHT, mean_HEIGHT)
df_pivot.columns = [f"{stat}_{vital}" for stat, vital in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Ensure all vital-related variables exist in df_encoded, even if missing in df_pivot
for vital in vitals:
    for stat in ["min", "max", "mean"]:
        col_name = f"{stat}_{vital}"
        if col_name not in df_pivot.columns:
            df_pivot[col_name] = 0  # Fill missing columns with 0

# Merge with df_encoded
df_encoded = df_encoded.merge(df_pivot, on="ID", how="left")

# Replace NaN values in the new columns with 0
for vital in vitals:
    for stat in ["min", "max", "mean"]:
        col_name = f"{stat}_{vital}"
        df_encoded[col_name] = df_encoded[col_name].fillna(0)










# Load the anonymized blood pressure data
bp_df = pd.read_csv("data/anonymized_H43413_bp.csv")

# Ensure ID columns are of the same type
bp_df['ID'] = bp_df['ID'].astype(str)
df_encoded['ID'] = df_encoded['ID'].astype(str)

# Compute blood pressure statistics for each patient
bp_stats = bp_df.groupby('ID').agg({
    'SYSTOLIC_BP': ['min', 'max', 'mean'],
    'DIASTOLIC_BP': ['min', 'max', 'mean']
}).reset_index()

# Rename columns for clarity
bp_stats.columns = ['ID', 'SYSTOLIC_BP_min', 'SYSTOLIC_BP_max', 'SYSTOLIC_BP_mean',
                    'DIASTOLIC_BP_min', 'DIASTOLIC_BP_max', 'DIASTOLIC_BP_mean']

# Merge with df_encoded
df_encoded = df_encoded.merge(bp_stats, on='ID', how='left')








# Load the lab test data
lab_data_path = "data/anonymized_h43413_labs.csv"
lab_data = pd.read_csv(lab_data_path)

# Define the tests of interest
lab_tests = ["CREATININE", "BMC_GLUCOSE", "BMC_ALT(SGPT)", "BMC_AST(SGOT)"]

# Filter only rows that match these lab tests
filtered_labs = lab_data[lab_data["RESULT_TEST_NM"].isin(lab_tests)]

# Ensure numeric values for results
filtered_labs["RESULT_VALUE_NUM"] = pd.to_numeric(filtered_labs["RESULT_VALUE_NUM"], errors="coerce")

# Load the encoded patient list
patient_ids = df_encoded["ID"].unique()

# Filter lab data to only include IDs present in df_encoded
filtered_labs = filtered_labs[filtered_labs["ID"].isin(patient_ids)]

# Compute min, max, mean for each lab test per patient
lab_summary = (
    filtered_labs.groupby(["ID", "RESULT_TEST_NM"])["RESULT_VALUE_NUM"]
    .agg(["min", "max", "mean"])
    .reset_index()
)

# Reshape data to have separate columns for each test
lab_summary = lab_summary.pivot(index="ID", columns="RESULT_TEST_NM", values=["min", "max", "mean"])

# Flatten multi-index column names
lab_summary.columns = [f"{stat}_{test}" for stat, test in lab_summary.columns]
lab_summary.reset_index(inplace=True)

# Merge the computed lab stats back into df_encoded
df_encoded = df_encoded.merge(lab_summary, on="ID", how="left")


# Load the Problem List sheet from the Excel file
problem_list_df = pd.read_excel("data/anonymized_H-43413 Data Add On.xlsx", sheet_name="Problem List")

# Ensure ID columns are of the same type
problem_list_df['ID'] = problem_list_df['ID'].astype(str)
df_encoded['ID'] = df_encoded['ID'].astype(str)

# List of ICD-10 codes to check
icd10_codes = [
    "F17.200", "F20.9", "F39", "F25.9", "F43.10", "F29", "F32.A", "F41.1", "F41.9", "F11.20", "F31.9", "F22", 
    "F25.0", "F20.0", "F33.1", "F19.10", "F10.20", "F10.10", "F43.20", "F14.10", "F32.9", "F17.210", "F14.20", 
    "F99", "F25.1", "F12.10", "F19.90", "F90.9", "F33.9", "F11.21", "F33.3", "F10.21", "F11.90", "F12.20", 
    "F43.21", "F19.20", "F20.3", "F10.11", "F41.8", "F41.0", "F33.2", "F20.89", "F34.1"
]

# Initialize binary columns in df_encoded
df_encoded[icd10_codes] = 0

# Map ICD-10 codes to binary variables
for index, row in problem_list_df.iterrows():
    patient_id = row['ID']
    icd_list = str(row['CURRENT ICD-10 LIST']).split(', ')
    
    if patient_id in df_encoded['ID'].values:
        for icd_code in icd10_codes:
            if icd_code in icd_list:
                df_encoded.loc[df_encoded['ID'] == patient_id, icd_code] = 1

def update_primary_race(df):
    # Set 'PRIMARY_RACE_Asian' to 1 where 'PRIMARY_RACE_Chinese' is 1
    df.loc[df['PRIMARY_RACE_Chinese'] == 1, 'PRIMARY_RACE_Asian'] = 1
    
    # Set 'PRIMARY_RACE_Native Hawaiian / Pacific Islander' to 1 if certain conditions are met
    df.loc[df['PRIMARY_RACE_Native Hawaiian / Other Pacific Islander'] == 1, 'PRIMARY_RACE_Native Hawaiian / Pacific Islander'] = 1
    df.loc[df['PRIMARY_RACE_Other Pacific Islander'] == 1, 'PRIMARY_RACE_Native Hawaiian / Pacific Islander'] = 1
    
    # Set 'PRIMARY_RACE_American Indian / Native American' to 1 if 'PRIMARY_RACE_American Indian or Alaskan Native' is 1
    df.loc[df['PRIMARY_RACE_American Indian or Alaskan Native'] == 1, 'PRIMARY_RACE_American Indian / Native American'] = 1
    
    # Set 'PRIMARY_RACE_Middle Eastern' to 1 if 'PRIMARY_ETHNICITY_Middle Eastern' is 1
    df.loc[df['PRIMARY_ETHNICITY_Middle Eastern'] == 1, 'PRIMARY_RACE_Middle Eastern'] = 1
    
    # Set 'PRIMARY_RACE_White' to 1 if any of the specified ethnicity columns are 1
    df.loc[df[['PRIMARY_ETHNICITY_Russian', 'PRIMARY_ETHNICITY_American', 'PRIMARY_ETHNICITY_European']].sum(axis=1) > 0, 'PRIMARY_RACE_White'] = 1
    
    # Set 'PRIMARY_RACE_Asian Indian' to 1 if 'PRIMARY_ETHNICITY_Asian Indian' is 1
    df.loc[df['PRIMARY_ETHNICITY_Asian Indian'] == 1, 'PRIMARY_RACE_Asian Indian'] = 1
    
    # Set 'PRIMARY_RACE_Black / African American' to 1 if any of the specified ethnicity columns are 1
    df.loc[df[['PRIMARY_ETHNICITY_African', 'PRIMARY_ETHNICITY_African American', 'PRIMARY_ETHNICITY_Cape Verdean', 'PRIMARY_ETHNICITY_Caribbean Islander', 'PRIMARY_ETHNICITY_Haitian', 'PRIMARY_ETHNICITY_Middle Eastern or North African']].sum(axis=1) > 0, 'PRIMARY_RACE_Black / African American'] = 1
    
    # Set 'PRIMARY_RACE_Asian' to 1 if any of the specified ethnicity columns are 1
    df.loc[df[['PRIMARY_ETHNICITY_Cambodian', 'PRIMARY_ETHNICITY_Chinese', 'PRIMARY_ETHNICITY_Filipino', 'PRIMARY_ETHNICITY_Japanese', 'PRIMARY_ETHNICITY_Korean', 'PRIMARY_ETHNICITY_Laotian', 'PRIMARY_ETHNICITY_Vietnamese']].sum(axis=1) > 0, 'PRIMARY_RACE_Asian'] = 1
    
    # Set 'PRIMARY_RACE_Unknown' based on the presence of any PRIMARY_RACE_ variables
    race_columns = [col for col in df.columns if col.startswith('PRIMARY_RACE_') and col != 'PRIMARY_RACE_Unknown']
    df['PRIMARY_RACE_Unknown'] = (df[race_columns].sum(axis=1) == 0).astype(int)
    
    # Set 'PRIMARY_ETHNICITY_Unknown' based on the presence of any PRIMARY_ETHNICITY_ variables
    ethnicity_columns = [col for col in df.columns if col.startswith('PRIMARY_ETHNICITY_') and col != 'PRIMARY_ETHNICITY_Unknown']
    df['PRIMARY_ETHNICITY_Unknown'] = (df[ethnicity_columns].sum(axis=1) == 0).astype(int)
    
    # Drop the specified columns
    df.drop(columns=['PRIMARY_RACE_Chinese', 'PRIMARY_RACE_Other Pacific Islander', 'PRIMARY_RACE_Native Hawaiian / Other Pacific Islander', 'PRIMARY_RACE_American Indian or Alaskan Native'], inplace=True)
    
    return df

# Apply the function to df_encoded
df_encoded = update_primary_race(df_encoded)


print(df_encoded.head())


C:\Users\khang\AppData\Local\Temp\ipykernel_15100\4107370961.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_labs["RESULT_VALUE_NUM"] = pd.to_numeric(filtered_labs["RESULT_VALUE_NUM"], errors="coerce")


      ID  demo_age  RPL_THEME1  GENDER_F  GENDER_M  GENDER_U  \
0  10000        49        0.49      True     False     False   
1  10001        67        0.75      True     False     False   
2  10002        80        0.79      True     False     False   
3  10004        83        0.55      True     False     False   
4  10005        82        0.66     False      True     False   

  PRIMARY_RACE_American Indian / Native American PRIMARY_RACE_Asian  \
0                                          False              False   
1                                          False              False   
2                                          False              False   
3                                          False              False   
4                                          False              False   

  PRIMARY_RACE_Asian Indian PRIMARY_RACE_Black / African American  ...  \
0                     False                                     1  ...   
1                     False             

C:\Users\khang\AppData\Local\Temp\ipykernel_15100\4107370961.py:333: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[df['PRIMARY_RACE_Chinese'] == 1, 'PRIMARY_RACE_Asian'] = 1
C:\Users\khang\AppData\Local\Temp\ipykernel_15100\4107370961.py:336: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[df['PRIMARY_RACE_Native Hawaiian / Other Pacific Islander'] == 1, 'PRIMARY_RACE_Native Hawaiian / Pacific Islander'] = 1
C:\Users\khang\AppData\Local\Temp\ipykernel_15100\4107370961.py:340: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with

In [4]:
import pandas as pd

# File paths
file_paths = [
    "data/anonymized_H-43413 Amb Med Data.xlsx",
    "data/anonymized_H43413_vitals.csv",
    "data/anonymized_H-43413 Qualifying Encounters.xlsx",
    "data/anonymized_h43413_labs.csv",
    "data/anonymized_H-43413 Data Add On.xlsx",
    "data/anonymized_H43413_bp.csv"
]

# Load df_encoded (Assuming df_encoded is already loaded)
df_encoded_ids = set(df_encoded['ID'].astype(str))  # Convert to string for consistency

# Function to get valid IDs from a spreadsheet
def get_valid_ids(file_path):
    try:
        
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, usecols=['ID'])  # Load only ID column
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path, usecols=['ID'])  # Load only ID column
        else:
            print(f"Unsupported file format: {file_path}")
            
            return set()
        # Load spreadsheet (CSV or Excel)
        if file_path ==  "data/anonymized_H-43413 Data Add On.xlsx":
            df = pd.read_excel("data/anonymized_H-43413 Data Add On.xlsx", sheet_name="Problem List", usecols=['ID'])
        

        return set(df['ID'].dropna().astype(str))  # Convert to string and remove NaNs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return set()

# Find IDs that exist in all spreadsheets
valid_ids = df_encoded_ids  # Start with df_encoded IDs
for file in file_paths:
    valid_ids &= get_valid_ids(file)  # Keep only IDs present in all files

# Filter df_encoded to keep only rows where ID is in valid_ids
df_encoded = df_encoded[df_encoded['ID'].astype(str).isin(valid_ids)]

# Convert boolean values to integers (0 and 1)
df_encoded = df_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))


C:\Users\khang\AppData\Local\Temp\ipykernel_15100\3294417873.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_encoded = df_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))


In [3]:
df_encoded.to_pickle("df_encoded.pkl")

In [4]:
print(df_encoded.columns.tolist())

['ID', 'demo_age', 'RPL_THEME1', 'GENDER_F', 'GENDER_M', 'GENDER_U', 'PRIMARY_RACE_American Indian / Native American', 'PRIMARY_RACE_Asian', 'PRIMARY_RACE_Asian Indian', 'PRIMARY_RACE_Black / African American', 'PRIMARY_RACE_Hispanic or Latino', 'PRIMARY_RACE_Middle Eastern', 'PRIMARY_RACE_Native Hawaiian / Pacific Islander', 'PRIMARY_RACE_Other', 'PRIMARY_RACE_Unknown', 'PRIMARY_RACE_White', 'LANGUAGE_Afrikaans', 'LANGUAGE_Albanian', 'LANGUAGE_American Sign Language', 'LANGUAGE_American Sign language & Certified Deaf Interpreter', 'LANGUAGE_Amharic / Ethiopia', 'LANGUAGE_Arabic', 'LANGUAGE_Bassa / Liberia', 'LANGUAGE_Bengali / Hindi / Urdu', 'LANGUAGE_Bosnian / Croatian / Yulo', 'LANGUAGE_Brazilian Portuguese', 'LANGUAGE_Cape Verdean / Port Creole', 'LANGUAGE_Chinese / Cantonese', 'LANGUAGE_Chinese / Mandarin', 'LANGUAGE_English', 'LANGUAGE_French', 'LANGUAGE_Fulani / Cameroon', 'LANGUAGE_German', 'LANGUAGE_Haitian Creole', 'LANGUAGE_Italian', 'LANGUAGE_Kirundi / Burundi', 'LANGUAGE_K